In [2]:
#
#Tweets Machine Learning Coursework 
#COMP3222
#

# To support both python 2 and 3
from __future__ import division, print_function, unicode_literals

# Common imports
import os
import numpy as np
import pandas as pd

# These two lines are required to use Tensorflow 1
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

# To plot nice figures
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

# Clear tensorflow's and reset seed
def reset_graph(seed=None):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)



Instructions for updating:
non-resource variables are not supported in the long term


In [3]:
#get data
def getData(file_name):
    dict_data = {}
    data_file = open(file_name, "r", encoding="utf8")
    raw_data_txt = data_file.readlines()
    raw_data = []
    for data in raw_data_txt:
        raw_data.append(data.split("\t"))
    raw_data.pop(0)

    target_text = [data[6] for data in raw_data]
    dict_data['text'] = np.array([tweet[1] for tweet in raw_data])
    dict_data['imageIds'] = np.array([tweet[3] for tweet in raw_data])
    dict_data['timestamp'] = np.array([tweet[5] for tweet in raw_data])
    dict_data['label'] = np.array([1 if target=='real' or target =='real\n' else 0 for target in target_text])

    return dict_data

df_train = pd.DataFrame.from_dict(getData("mediaeval-2015-trainingset.txt"))
df_test = pd.DataFrame.from_dict(getData("mediaeval-2015-testset.txt"))
print("Done")


Done


In [113]:
#save file pre-processing to csv
# df_test.to_csv('test_v2.2.1.csv')
# df_train.to_csv('train_v2.2.1.csv')

In [7]:
df_train = pd.read_csv("train_v2.2.0.csv")
df_train = df_train.iloc[:, 1:]
df_train
df_train.text = df_train.text.astype(str)

In [6]:
df_test = pd.read_csv("test_v2.2.0.csv")
df_test = df_test.iloc[:, 1:]
df_test
df_test.text = df_test.text.astype(str)

In [8]:
#Find all the different languages
#https://meta.wikimedia.org/wiki/Template:List_of_language_names_ordered_by_code conversion of symbol to langauge table
from langdetect import detect

def language_detect(data_frame):
    languages = []
    for tweet in data_frame['text'].values:
        try:
            languages.append(detect(tweet))
        except:
            languages.append('en')

    data_frame.insert(3, "language", languages, True)

    languages_seen = set(languages)
    print("Languages seen: ", languages_seen)
    return data_frame

In [9]:
def retweet_detection(data_frame):
    retweets = [1 if tweet.count("RT") >= 1 else 0 for tweet in data_frame["text"].values]
    data_frame.insert(4, "retweet", retweets, True)
    return data_frame

In [24]:
from deep_translator import GoogleTranslator

def translate(data_frame):
    translate_text = {}
    for i in range(len(data_frame)):
        if(data_frame.iloc[i]['language'] != 'en'):
            translate_text[data_frame.iloc[i]['text']] = GoogleTranslator(source='auto', target='en').translate(data_frame.iloc[i]['text'])
            #print(i, data_frame.iloc[i]['language'], GoogleTranslator(source='auto', target='en').translate(df_train.iloc[i]['text']))

    data_frame["text"].replace(translate_text, inplace=True)
    return data_frame

In [7]:
df_train

,text,imageIds,timestamp,language,num_emoji,num_hashtags,profane,retweet,label
0,"Do you remember the movie: ""The Day After Tomo...",sandyA_fake_46,Mon Oct 29 22:34:01 +0000 2012,es,0,1,0,0,0
1,@milenagimon: Look at Sandy in NY! Tremendous ...,sandyA_fake_09,Mon Oct 29 19:11:23 +0000 2012,es,0,0,0,1,0
2,"Good the photo of Hurricane Sandy, it reminds ...",sandyA_fake_09,Mon Oct 29 18:11:08 +0000 2012,es,0,2,0,0,0
3,Scary shit #hurricane #NY http://t.co/e4JLBUfH,sandyA_fake_29,Mon Oct 29 19:15:33 +0000 2012,en,0,2,1,0,0
4,My fave place in the world #nyc #hurricane #sa...,sandyA_fake_15,Mon Oct 29 20:46:02 +0000 2012,en,1,4,0,0,0
...,...,...,...,...,...,...,...,...,...
14478,@BobombDom *slaps TweetDeck with the PigFish h...,pigFish_01,Tue Mar 11 03: 48: 36 +0000 2014,en,0,0,0,0,0
14479,New Species of Fish found in Brazil or just Re...,pigFish_01,Mon Mar 10 18: 09: 26 +0000 2014,en,0,0,0,0,0
14480,What do we call this? #pigFISH http: \/\/t.co\...,pigFish_01,Mon Mar 10 10: 59: 45 +0000 2014,en,0,1,0,0,0
14481,Pigfish? And after the shark there is the pork...,pigFish_01,Sun Mar 09 20: 07: 10 +0000 2014,it,0,0,0,0,0


In [11]:
import spacy
from profanity_filter import ProfanityFilter

def profanity_detection(data_frame):
    nlp = spacy.load('en_core_web_sm')
    profanity_filter = ProfanityFilter(nlps={'en': nlp})  # reuse spacy Language (optional)
    nlp.add_pipe(profanity_filter.spacy_component, last=True)

    doc = nlp('This is fuck shit!')

    doc._.is_profane
    profanity_check = []
    count = 0
    for tweet in data_frame['text'].values:
        text = nlp(tweet)
        if(text._.is_profane):
            profanity_check.append(1)
        else: 
            profanity_check.append(0)
        
        #print(count, text._.is_profane)
        count += 1
    
    data_frame.insert(4, "profane", profanity_check, True)
    return data_frame

In [12]:
def numofhashtags(data_frame):
    num_hashtags = []
    for i in range(len(data_frame)):
        num_hashtags.append(data_frame.iloc[i]['text'].count("#"))
    #maybe count number of punctuation as well.
    data_frame.insert(4, "num_hashtags", num_hashtags, True)
    return data_frame


In [13]:
import emoji
def emoji_detect(data_frame):
    num_emojis = []
    for i in range(len(data_frame)):
        emoji_count = emoji.emoji_count(data_frame.iloc[i]['text'])
        num_emojis.append(emoji_count)
    
    data_frame.insert(4, "num_emoji", num_emojis, True)
    return data_frame


In [137]:
import regex as re

def parseText(data_frame):
    parsed_text = {}
    for i in range(len(data_frame)):
        parsed = ' '.join(re.sub("(@[A-Za-z0-9_]+)|((\\\/\\\/\w+.co\S+)|(\w+:\/\/\S+))|([^0-9A-Za-z \t])"," ",data_frame.iloc[i]['text']).split())
        parsed_text[data_frame.iloc[i]['text']] = parsed.lower()
        #print(i, parsed)
    data_frame["text"].replace(parsed_text, inplace=True)
    return data_frame
    

In [21]:
#based of https://craft.co/cnn/competitors
#
def news_companies(data_frame):
    news_companies =["cnn", "bbc", "nbc", "new york times", "wall street journal", " ap ", "fox", "cnbc", "daily mail"]
    news_found = []
    for tweet in data_frame['text'].values:
        tweet = tweet.lower()
        if(any(news in tweet for news in news_companies) or (tweet.find('ap') == 0) or (tweet.find('ap') == len(tweet)-2) ) :
            news_found.append(1)
        else:
            news_found.append(0)
    data_frame.insert(4, "news_company_found", news_found, True)
    return data_frame

In [16]:
#location detector too much noise picked up

from geopy.geocoders import Nominatim
def location_detector():
    geolocator = Nominatim(user_agent="COMP322-MLT")
    x = "My fave place in the world nyc hurricane sandy times sqaure"
    x = x.split(' ')
    for i in range(len(x)):
        try:
            location = geolocator.geocode(x[i])
            print(location.address)
        except:
            print("Notfound")

In [25]:
#Run hashtags if first time running on machine

from nltk.corpus import stopwords
#nltk.download('stopwords')
#nltk.download('punkt')
from nltk.tokenize import word_tokenize

def stopWordRemoval(data_frame):
    stop_words = set(stopwords.words('english'))
    filtered_text = {}
    for i in range(len(data_frame)):
        word_tokens = word_tokenize(data_frame.iloc[i]['text'])
        filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words]
        new_sentence = " ".join(filtered_sentence)
        filtered_text[data_frame.iloc[i]['text']] = new_sentence
        #print(i, new_sentence)
    data_frame["text"].replace(filtered_text, inplace=True)
    return data_frame

In [2]:
from nltk.corpus import stopwords
from collections import defaultdict
from nltk.tokenize import word_tokenize
from nltk import pos_tag, download
# download('averaged_perceptron_tagger')
# download('stopwords')
# download('punkt')

def POS_tagger(data_frame):
    pos_data_tags = []
    for i in range(len(data_frame)):
        try:
            word_tokens = word_tokenize(data_frame.iloc[i]['text'])
        except:
            print(data_frame.iloc[i])
        pos = pos_tag(word_tokens)
        exclusive_pos  = " ".join(x[1] for x in pos)
        pos_data_tags.append(exclusive_pos)
        # pos_data_tags.append(pos)
    data_frame.insert(1, "pos", pos_data_tags, True)
    return data_frame


In [22]:
from spellchecker import SpellChecker
def misspelling_detection(data_frame):
    spell = SpellChecker()
    misspelled_count = []
    for i in range(len(data_frame)):
        word_list = word_tokenize(data_frame.iloc[i]['text'])
        misspelled = spell.unknown(word_list)
        misspelled_count.append(len(misspelled))
        #print(i, len(misspelled), misspelled))
    data_frame.insert(4, "misspellings", misspelled_count, True)
    return data_frame

In [17]:
from collections import defaultdict, Counter


fake_words = defaultdict(lambda: 0)
real_words = defaultdict(lambda: 0)
total_words = defaultdict(lambda: 0)

for i in range(len(df_train)):
    text = str(df_train.iloc[i]['text']).lower()
    text = text.split()
    if(df_train.iloc[i]['label'] == 0):
        for j in range(len(text)):
            fake_words[text[j]] += 1
            total_words[text[j]] += 1
    else:
        for j in range(len(text)):
            real_words[text[j]] += 1
            total_words[text[j]] += 1


real_words = dict(sorted(real_words.items(), reverse=True, key=lambda item: item[1]))
fake_words = dict(sorted(fake_words.items(), reverse=True, key=lambda item: item[1]))
total_words = dict(sorted(total_words.items(), reverse=False, key=lambda item: item[1]))

words = list(total_words.keys())
for w in words:
    if(total_words[w] < 100):
        del total_words[w]
        try:
            del real_words[w]
        except:
            pass
        try:
            del fake_words[w]
        except:
            pass
    
rw = Counter(real_words)
fw = Counter(fake_words)
fake_res = fw - rw
real_res = rw - fw

real_res = {k: real_res[k]/total_words[k] for k in total_words.keys() & real_res}
fake_res = {k: fake_res[k]/total_words[k] for k in total_words.keys() & fake_res}

real_res = dict(sorted(real_res.items(), reverse=True, key=lambda item: item[1]))
fake_res = dict(sorted(fake_res.items(), reverse=True, key=lambda item: item[1]))


sig_real_words = list(k for k, v in real_res.items() if v > 0.5)
sig_fake_words = list(k for k, v in fake_res.items() if v > 0.5)

def significance(data_frame, sig_words, clmn_name):
    sig_found = []
    for tweet in data_frame['text'].values:
        try:
            tweet = tweet.lower()
            if(any(word in tweet for word in sig_words)):
                sig_found.append(1)
            else:
                sig_found.append(0)
        except:
            sig_found.append(0)
    data_frame.insert(4, clmn_name, sig_found, True)
    return data_frame

In [150]:
import time 
def epochTime(data_frame):

    month_dict = {
        'Jan': '01',
        'Feb': '02',
        'Mar': '03',
        'Apr': '04',
        'May': '05',
        'Jun': '06',
        'Jul': '07',
        'Aug': '08',
        'Sep': '09',
        'Oct': '10',
        'Nov': '11',
        'Dec': '12'
    }
    time_epoch = {}
    for i in range(len(data_frame)):
        str_time = str(data_frame.iloc[i]['timestamp'])
        str_time = str_time.split()
        for j in range(len(str_time)):
            if(str_time[j] in month_dict.keys()):
                str_time[j] = month_dict[str_time[j]]
                break

        str_time = str_time[2] + '.' + str_time[1] +'.' +str_time[len(str_time)-1]+' '+str_time[3][:2] + ':' +str_time[3][3:5] +':'+ str_time[3][6:]
        str_time
        try:
            pattern = '%d.%m.%Y %H:%M:%S'
            epoch = int(time.mktime(time.strptime(str_time, pattern)))
        except:
            epoch = -1
        time_epoch[data_frame.iloc[i]['timestamp']] = epoch
        
    data_frame["timestamp"].replace(time_epoch, inplace=True)
    return data_frame

In [ ]:
from nltk import ngrams

def n_grams(data_frame, n):
    a = []
    for i in range(len(data_frame)):
        sentence = data_frame.iloc[i]['text']

        ng = ngrams(sentence.split(), n)
        x = []
        for grams in ng:
            x.append(grams)
        a.append(x)
    data_frame.insert(3, ng+"-gram", a, True)

In [30]:
def pre_process(df):
    # df = language_detect(df)
    # print("Language Detection Done")
    # df = translate(df)
    # print("Translation Done")
    # df = retweet_detection(df)
    # print("Retweet Detection Done")
    df = news_companies(df)
    print("News companies Detection Done")
    # df = numofhashtags(df)
    # print("Hashtag count Done")
    # df = emoji_detect(df)
    # print("Emoji count Done")
    # df = parseText(df)
    # print("Parsing Done")
    # df = stopWordRemoval(df)
    # print("Stopword removal Done")
    # df = misspelling_detection(df)
    # print("Misspelling Count Done")
    # # df = profanity_detection(df)
    # # print("Profanity Detection Done")
    # df = POS_tagger(df)
    # print("POS tagging done")
    # df = significance(df, sig_real_words, "significant_real")
    # print("Real Significant Words Detected")
    # df = significance(df, sig_fake_words, "significant_fake")
    # print("Fake Significant Words Detected")
    # df = epochTime(df)
    # print("Epoch Time Done")

    print("Preprocessing done")
    return df
#before passing the text as unigram .lower everything

In [28]:
df_train = pre_process(df_train)
df_train

Misspelling Count Done
Preprocessing done


,text,pos,imageIds,timestamp,misspellings,news_company_found,language,num_emoji,num_hashtags,profane,retweet,label
0,remember movie day tomorrow reminds happening ...,VB NN NN NN VBZ VBG NN NN,sandyA_fake_46,1351550041,0,0,es,0,1,0,0,0
1,look sandy ny tremendous image hurricane looks...,NN JJ RB JJ NN NN VBZ IN NN NN CD JJ NN,sandyA_fake_09,1351537883,1,0,es,0,0,0,1,0
2,good photo hurricane sandy reminds movie indep...,JJ NN NN JJ VBZ NN NN NN VBZ NN,sandyA_fake_09,1351534268,1,0,es,0,2,0,0,0
3,scary shit hurricane ny,JJ VBD NN NN,sandyA_fake_29,1351538133,0,0,en,0,2,1,0,0
4,fave place world nyc hurricane sandy statueofl...,JJ NN NN NN NN NN NN,sandyA_fake_15,1351543562,2,0,en,1,4,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
14478,slaps tweetdeck pigfish http,NNS VBP JJ NN,pigFish_01,-1,3,0,en,0,0,0,0,0
14479,new species fish found brazil really good phot...,JJ NNS VBP VBN IN RB JJ NN VBP NN,pigFish_01,-1,1,0,en,0,0,0,0,0
14480,call pigfish http,VB JJ NN,pigFish_01,-1,2,0,en,0,1,0,0,0
14481,pigfish shark pork fish http co hqzwghydef,JJ NN NN JJ NN NN NN,pigFish_01,-1,3,0,it,0,0,0,0,0


In [31]:
df_test = pre_process(df_test)
df_test

News companies Detection Done
Preprocessing done


,text,pos,imageIds,timestamp,news_company_found,language,profane,misspellings,num_emoji,label
0,kereeen rt eclipse iss,JJ NN NN NN,eclipse_01,1426844743,0,en,0,2,0,0
1,absolutely beautiful rt eclipse iss,RB JJ NN NN NN,eclipse_01,1426849442,0,en,0,1,0,0
2,eclipse iss 3 20 wow amazing,NN VBZ CD CD NN NN,eclipse_01,1426853406,0,en,0,0,0,0
3,eclipse iss,NN NN,eclipse_01,1426842761,0,en,0,0,0,0
4,eclipse seen iss something else divine creatio...,NN VBN JJ NN RB JJ NN NN,eclipse_01,1426873451,0,fr,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...
3776,zdf presenter confesses rigged video varoufaki...,NN NN NNS VBD NN NN NN JJ NN IN,varoufakis_1,1426744184,0,fr,0,2,0,0
3777,oh kleine liars zdf presenter confesses faked ...,UH NN NNS VBP NN NNS VBD JJ NN NN,varoufakis_1,1426744302,0,fr,0,2,0,0
3778,zdf program confirm varoufakis video montage,NN NN NN VBZ JJ NN,varoufakis_1,1426757001,0,es,0,2,0,0
3779,11 34 almost noon big confusion varoufakis vid...,CD CD RB RB JJ NN NN NN NN JJ NN NN NN,varoufakis_1,1426761320,0,de,0,4,2,0


In [8]:
df_test

,text,pos,imageIds,timestamp,news_company_found,language,profane,misspellings,num_emoji,label
0,kereeen rt eclipse iss,JJ NN NN NN,eclipse_01,1426844743,0,en,0,2,0,0
1,absolutely beautiful rt eclipse iss,RB JJ NN NN NN,eclipse_01,1426849442,0,en,0,1,0,0
2,eclipse iss 3 20 wow amazing,NN VBZ CD CD NN NN,eclipse_01,1426853406,0,en,0,0,0,0
3,eclipse iss,NN NN,eclipse_01,1426842761,0,en,0,0,0,0
4,eclipse seen iss something else divine creatio...,NN VBN JJ NN RB JJ NN NN,eclipse_01,1426873451,0,fr,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...
3776,zdf presenter confesses rigged video varoufaki...,NN NN NNS VBD NN NN NN JJ NN IN,varoufakis_1,1426744184,0,fr,0,2,0,0
3777,oh kleine liars zdf presenter confesses faked ...,UH NN NNS VBP NN NNS VBD JJ NN NN,varoufakis_1,1426744302,0,fr,0,2,0,0
3778,zdf program confirm varoufakis video montage,NN NN NN VBZ JJ NN,varoufakis_1,1426757001,0,es,0,2,0,0
3779,11 34 almost noon big confusion varoufakis vid...,CD CD RB RB JJ NN NN NN NN JJ NN NN NN,varoufakis_1,1426761320,0,de,0,4,2,0


In [9]:
def pull_features(data_frame):
    features = []
    for i in range(len(data_frame)):
        temp = []
        #temp.append(data_frame.iloc[i]['significant_fake'])
        #temp.append(data_frame.iloc[i]['significant_real'])
        temp.append(data_frame.iloc[i]['profane'])
        temp.append(data_frame.iloc[i]['misspellings'])
        #temp.append(data_frame.iloc[i]['num_emoji'])
        #temp.append(data_frame.iloc[i]['num_hashtags'])
        temp.append(data_frame.iloc[i]['news_company_found'])
        features.append(temp)
    features = np.array(features)
    return features

In [10]:
texts = df_train['text']
texts.append(df_test['text'])
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(texts)
df_train['text'] = tokenizer.texts_to_sequences(df_train['text'])
MAX_TWEET_LENGTH = 16

df_test['text'] = tokenizer.texts_to_sequences(df_test['text'])

biggest_token = max(tokenizer.index_word)

In [11]:
X_train_text = df_train.iloc[:]['text']
X_test_text = df_test.iloc[:]['text']

X_train_text = tf.keras.preprocessing.sequence.pad_sequences(X_train_text, maxlen=MAX_TWEET_LENGTH, padding='post', truncating='post')
X_test_text = tf.keras.preprocessing.sequence.pad_sequences(X_test_text, maxlen=MAX_TWEET_LENGTH, padding='post', truncating='post')

In [12]:
X_train_features = pull_features(df_train)
X_test_features = pull_features(df_test)

In [13]:
X_train_features.max()

10

In [17]:
feat_inputs = tf.keras.layers.Input(shape=(3,), name='features')
feat_embed = tf.keras.layers.Embedding(X_train_features.max()+1, 50, input_length=3, name='feature_embedding')(feat_inputs)
feat_conv = tf.keras.layers.Conv1D(128, 3, name='feature_conv')(feat_embed)
feat_pool = tf.keras.layers.GlobalAveragePooling1D(name='feature_pooling')(feat_conv)


text_input = tf.keras.layers.Input(shape=(MAX_TWEET_LENGTH,), name='tweet_input')
text_embed = tf.keras.layers.Embedding(biggest_token + 1, 50, input_length=MAX_TWEET_LENGTH, name='article_body_embedding')(text_input)
text_conv = tf.keras.layers.Conv1D(128, 8, name='article_body_conv')(text_embed)
text_pool = tf.keras.layers.GlobalAveragePooling1D(name='article_body_pooling')(text_conv)

concat = tf.keras.layers.concatenate([feat_pool, text_pool])
dense_10 = tf.keras.layers.Dense(75, activation='selu')(concat)
dense_5= tf.keras.layers.Dense(50, activation='selu')(dense_10)
dense_2= tf.keras.layers.Dense(20, activation='selu')(dense_5)
out_layer = tf.keras.layers.Dense(1, activation='sigmoid')(dense_2)
model = tf.keras.models.Model(inputs=[feat_inputs, text_input], outputs=out_layer)
model.summary()
model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
features (InputLayer)           [(None, 3)]          0                                            
__________________________________________________________________________________________________
tweet_input (InputLayer)        [(None, 16)]         0                                            
__________________________________________________________________________________________________
feature_embedding (Embedding)   (None, 3, 50)        550         features[0][0]                   
__________________________________________________________________________________________________
article_body_embedding (Embeddi (None, 16, 50)       454500      tweet_input[0][0]                
_______________________________________________________________________________________

In [18]:
y_train = df_train['label'].values
y_test = df_test['label'].values
callback = tf.keras.callbacks.EarlyStopping(monitor='val_acc', patience=5, mode='max', restore_best_weights=True)

history = model.fit([X_train_features, X_train_text], y_train, epochs=100, batch_size=128, validation_data=([X_test_features, X_test_text], y_test), callbacks=[callback])

Train on 14483 samples, validate on 3781 samples
Epoch 1/100
14483/14483 [==============================] - 2s 125us/sample - loss: 0.3740 - acc: 0.8271 - val_loss: 0.5166 - val_acc: 0.7903
Epoch 2/100
14483/14483 [==============================] - 2s 115us/sample - loss: 0.1742 - acc: 0.9323 - val_loss: 0.5401 - val_acc: 0.7982
Epoch 3/100
14483/14483 [==============================] - 2s 113us/sample - loss: 0.1216 - acc: 0.9553 - val_loss: 0.6049 - val_acc: 0.8373
Epoch 4/100
14483/14483 [==============================] - 2s 113us/sample - loss: 0.0982 - acc: 0.9639 - val_loss: 0.6729 - val_acc: 0.8228
Epoch 5/100
14483/14483 [==============================] - 2s 115us/sample - loss: 0.0856 - acc: 0.9673 - val_loss: 0.7783 - val_acc: 0.8146
Epoch 6/100
14483/14483 [==============================] - 2s 114us/sample - loss: 0.0796 - acc: 0.9698 - val_loss: 0.6948 - val_acc: 0.8471
Epoch 7/100
14483/14483 [==============================] - 2s 115us/sample - loss: 0.0746 - acc: 0.9721 -

In [19]:
import sklearn.metrics as skl_m

preds = model.predict([X_test_features, X_test_text])
print(model.evaluate([X_test_features, X_test_text], y_test))

y_pred = []
for i in range(len(preds)):
    if(preds[i]<0.5):
        y_pred.append(0)
    else:
        y_pred.append(1)
    
y_pred = np.array(y_pred)

f1 = skl_m.f1_score(y_test, y_pred, average='micro')
f1


[0.7705192790243767, 0.85136205]


0.8513620735255224